In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%sql
select * from workspace.silver.silvertable

OrderID,OrderDate,CustomerID,CustomerName,ProductID,ProductName,Quantity,SalesAmount,lastupdated
1001,2025-08-01,C001,John Doe,P001,Laptop,1,1200.0,2025-08-10T06:52:44.088Z
1002,2025-08-01,C002,Jane Smith,P002,Mouse,2,50.0,2025-08-10T06:52:44.088Z
1003,2025-08-02,C001,John Doe,P003,Keyboard,1,80.0,2025-08-10T06:52:44.088Z
1004,2025-08-03,C003,Mark Lee,P001,Laptop,1,1200.0,2025-08-10T06:52:44.088Z
1006,2025-08-06,C006,Emma White,P006,Webcam,1,90.0,2025-08-10T07:16:19.258Z
1007,2025-08-07,C002,Jane Smith,P002,Mouse,3,75.0,2025-08-10T07:16:19.258Z
1005,2025-08-04,C004,Suresh kumar,P004,Monitor,2,300.0,2025-08-10T07:16:19.258Z


## CUSTOMER DIMENSION TABLE

In [0]:
%sql
create or replace table workspace.gold.dimcustomertable
as
with remove_dup
as
(select distinct(customerid),customername from workspace.silver.silvertable)
select *, row_number() over(order by customerid) as dimcustomerkey
from remove_dup


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from workspace.gold.dimcustomertable

customerid,customername,dimcustomerkey
C001,John Doe,1
C002,Jane Smith,2
C003,Mark Lee,3
C004,Suresh kumar,4
C006,Emma White,5


## PRODUCT DIMENSION TABLE

In [0]:
%sql
create or replace table workspace.gold.dimproducttable
as
with remove_dup
as
(select distinct(productid),productname from workspace.silver.silvertable)
select * , row_number() over(order by productid) as dimproductkey
from remove_dup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from workspace.gold.dimproducttable

productid,productname,dimproductkey
P001,Laptop,1
P002,Mouse,2
P003,Keyboard,3
P004,Monitor,4
P006,Webcam,5


## SALES DIMENSION TABLE

In [0]:
%sql
create or replace table workspace.gold.dimsalestable
as
with remove_dup
as
(select * from workspace.silver.silvertable)
select * , row_number() over(order by OrderID) as dimsaleskey
from remove_dup



num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from workspace.gold.dimsalestable

OrderID,OrderDate,CustomerID,CustomerName,ProductID,ProductName,Quantity,SalesAmount,lastupdated,dimsaleskey
1001,2025-08-01,C001,John Doe,P001,Laptop,1,1200.0,2025-08-10T06:52:44.088Z,1
1002,2025-08-01,C002,Jane Smith,P002,Mouse,2,50.0,2025-08-10T06:52:44.088Z,2
1003,2025-08-02,C001,John Doe,P003,Keyboard,1,80.0,2025-08-10T06:52:44.088Z,3
1004,2025-08-03,C003,Mark Lee,P001,Laptop,1,1200.0,2025-08-10T06:52:44.088Z,4
1005,2025-08-04,C004,Suresh kumar,P004,Monitor,2,300.0,2025-08-10T07:16:19.258Z,5
1006,2025-08-06,C006,Emma White,P006,Webcam,1,90.0,2025-08-10T07:16:19.258Z,6
1007,2025-08-07,C002,Jane Smith,P002,Mouse,3,75.0,2025-08-10T07:16:19.258Z,7


## FACT TABLE

In [0]:
%sql
create or replace table workspace.gold.facttable
as
select
C.dimcustomerkey,
P.dimproductkey,
SK.dimsaleskey,
S.quantity,
S.salesamount
from workspace.silver.silvertable as S
left join workspace.gold.dimcustomertable as C
on S.customerid = C.customerid
left join workspace.gold.dimproducttable as P
on S.productid = P.productid
left join workspace.gold.dimsalestable as SK
on S.orderid = SK.orderid



num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from workspace.gold.facttable

dimcustomerkey,dimproductkey,dimsaleskey,quantity,salesamount
1,1,1,1,1200.0
2,2,2,2,50.0
1,3,3,1,80.0
3,1,4,1,1200.0
5,5,6,1,90.0
2,2,7,3,75.0
4,4,5,2,300.0
